In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
nltk.download('punkt')      # tokenizer package that helps divide a large piece of text into sentences or words or abbreviations
#punkt is also nltk's default word tokenizer package

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-04-20 06:22:24--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.08s   

2023-04-20 06:22:24 (58.1 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [3]:
text_df = pd.read_csv('bbc_text_cls.csv')
text_df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [18]:
text_df['labels'].unique()

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [8]:
text_df['text'].sample(10)

1063    MPs' murder sentence concern\n\nMurder sentenc...
398     Indian oil firm eyes Yukos assets\n\nIndia's b...
389     Wal-Mart to pay $14m in gun suit\n\nThe world'...
363     Durex maker SSL awaits firm bid\n\nUK condom m...
1360    Greek pair set for hearing\n\nKostas Kenteris ...
542     DVD review: Harry Potter and the Prisoner of A...
209     Gold falls on IMF sale concerns\n\nThe price o...
1889    Kenyan school turns to handhelds\n\nAt the Mbi...
1861    Movie body hits peer-to-peer nets\n\nThe movie...
1497    Candela completes Bolton switch\n\nBolton boss...
Name: text, dtype: object

In [32]:
# word-to-index mapping, necessary while making count vectorizer function later, because we cannot make a tf idf thing without knowing the counts of each token in the documents, which is what count vectorizer does
word_to_id = {}
id = 0
tokenized_text = []         # to store the list of tokenized documents i.e. storing those document names which have already undergone the tokenization process
for tokens in text_df['text']:
  token = word_tokenize(tokens.lower())
  document_to_id = []
  for t in token:
    if t not in word_to_id:
      word_to_id[t] = id                  # t is the word, id is the number which we will use for our reference further
      id += 1
    
    document_to_id.append(word_to_id[t])              # saving each word as index in a separate list for later usage
  tokenized_text.append(document_to_id)              # all the tokenized text now stored in separate lists
    

In [33]:
# reverse index-to-word mapping
rev_map = {value:key for key, value in word_to_id.items()}               # therefore here, the key is the word/token itself and the value is the number we allocated to it during making the word-to-index mapping
# rev_map
# we can also store all the key value pairs using lists, store the words using the value from word_to_id dict in the form of the array's indexes

In [11]:
# each text is considered as a separate document 
num_of_docs = len(text_df['text'])
num_of_docs

2225

In [12]:
# number of words to be considered
num_of_words = len(word_to_id)
num_of_words

38787

In [14]:
# make a dense matrix(sparse matrix would be better), taking the number of docs and number of words as the dimensions for the matrix
array = np.zeros((num_of_docs, num_of_words))         

In [15]:
# now we populate our newly created array using enumerate
for i, document_to_id in enumerate(tokenized_text):               #tokenized text is the whole thing, first take each document separately using document_to_id
  for j in document_to_id:
    array[i,j] += 1 

In [31]:
document_freq = np.sum(array>0, axis = 0)            # need to find out how much frequency for each word in each doc, using axis = 0 it will give us the sum of word occurrences (if value exists, it is true, which is basically evaulated using array>0)
idf = np.log(num_of_docs/document_freq)                 # this is inverse document frequency formula for a term

In [38]:
tf_idf = array*idf

In [ ]:
np.random.seed(123)               # its a pseudo-random generator generating random numbers that arent truly random because they are determined by algorithm

In [55]:
test_document = np.random.choice(num_of_docs)
row = text_df.iloc[test_document]                # if we take round brackets instead of square here, it assumes the given parameter to be axis, not index
print("label= ", row['labels'])
print("text= ", row['text'].split('\n', 1)[0])          # the text in df contains some \n characters, so we split the sentences in that and retrieve the first sentence(title) from that location
print('top 5 terms:')

scores = tf_idf[test_document]          # we need tf_idf score of test_document
indices = (-scores).argsort()

for j in indices[:10]:
  if rev_map[j].isnumeric():
    rev_map.pop[j]
  # for i in rev_map[j]:
  #   if i.isdigit():
  #     rev_map[j].replace(i, ' ')
  print(rev_map[j])

label=  business
text=  Cannabis hopes for drug firm
top 5 terms:
riyadh
essentially
follow-up
malta
year-ago
£5.38bn
addressed
looked
fifth-largest
panicked


using scipy's CSR matrix, and another part to initialize counts using count vectorizer 